In this lesson, we trained a linear neural network to distinguish background green pixels from other foreground pixels. Do you think the network will converge to the same weights every time it is run? Try it and see. Is there any property of the weights that seems consistent from one run to another?

In [1]:
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/greenML.png

--2020-06-26 11:39:38--  https://github.com/mlittmancs/great_courses_ml/raw/master/greenML.png
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/greenML.png [following]
--2020-06-26 11:39:39--  https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/greenML.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1190198 (1.1M) [image/png]
Saving to: ‘greenML.png’

greenML.png         100%[===================>]   1.13M  --.-KB/s    in 0.09s   

2020-06-26 11:39:39 (12.8 MB/s) - ‘greenML.png’ saved [1190198/1190198]



Here, we download the image.

In [2]:
import numpy as np
from keras.preprocessing import image

img = image.load_img("greenML.png")

Using TensorFlow backend.


Below we trim the edges of the image.

In [3]:
arr = image.img_to_array(img)
# Trim off edges
arr = arr[:697,:]

In this example, we isolate out the bacground and convert it to a dataset of positive examples, `yesList`.

In [4]:
# background
tmp = arr[:,:360]

yesList = np.reshape(tmp,(-1,3))

Below we print the number of values in each dimension of `tmp`.

Now we'll isolate out the foreground and make a dataset of foreground pixels called `noList`.

In [5]:
# foreground
tmp = arr[30:,547:620]

noList = np.reshape(tmp,(-1,3))

We finalize our dataset here, with a variable `alldat` which cointains our list of pixels, and `labs` which is our list of labels for each pixel, `0` for green background pixels and `1` for foreground pixels

In [6]:
# Build a list of pixels for both positive and negative examples.
alldat = np.concatenate((yesList,noList))
 
# labels
labs = np.concatenate((np.ones(len(yesList)), np.zeros(len(noList))))

Here, we'll build a classifier to separate the background from the foreground

We define a `loss` function.  The `loss` takes in our data, `alldat`, our labels `labs`, and our current weights, `w`.  We will make adjustments to `w` based on this loss function.  To make a prediction, we will multiply `w` by `alldat`, and pass it through a sigmoid function to get our predictions, `y`.  We will then compare our predictions `y` to their true labels `labs` using a squared loss, the sum of the squared difference between `labs`, our true labels, and `y` our predicted values

In [7]:
# Add an additional column to the data corresponding to 
#  the offset parameter.
alldat = np.concatenate((alldat,np.ones((len(alldat),1))),1)

# Compute the loss of the rule specified by weights w with respect
#  to the data alldat labeled with labs
def loss(w, alldat, labs):
  # Compute a weighted sum for each instance
  h = np.matmul(alldat,w)
  # transform the sum using the sigmoid function
  y = 1/(1 + np.exp(-h))
  # take the difference between the labels and the output of the 
  #  sigmoid, squared, then sum up over all instances to get the 
  #  total loss.
  loss = np.sum((labs - y)**2)
  return(loss)

Now we'll train the model, by creating a `fit` function to fit the model to the data.  We will update the weights through gradient descent. 

In [16]:
def fit(w,alldat,labs):
  # alpha represents how big of a step we’ll
  #  be taking in the direction of the derivative.
  #  It’s called the learning rate.
  alpha = 0.1

  # We'll stop searching when we're at a (near) local min
  done = False
  while not done:
    # Every 100 iterations or so, let’s
    #  take a peek at the weights, the learning
    #  rate, and the current loss
    if np.random.random() < 0.01: print(w, alpha, loss(w,alldat,labs))
    # The next few lines compute the gradient
    #  of the loss function. The details aren’t
    #  important right now.
    # delta_w is the change in the weights
    #  suggested by the gradient
    h = np.matmul(alldat,w)
    y = 1/(1 + np.exp(-h))
    delta_w = np.add.reduce(np.reshape((labs-y) * np.exp(-h)*y**2,(len(y),1)) * alldat)
    # if we take a step of size alpha and update
    #  the weights, we’ll get new weights neww.
    current_loss = loss(w,alldat,labs)
    alpha *= 2
    neww = w + alpha* delta_w
    while loss(neww,alldat,labs) >= current_loss and not done:
      alpha /= 2
      if alpha*max(abs(delta_w)) < 0.0001: 
        done = True
        print(alpha,delta_w)
      else: neww = w + alpha* delta_w
    if not done: w = neww
  return(w)

w = np.random.random(4)
w = fit(w,alldat,labs)
print(w)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in exp
  # This is added back by InteractiveShellApp.init_path()


[-0.71973051  0.14904301 -0.4152915  -0.329474  ] 1.220703125e-05 451.06624660571356
[-0.72051788  0.14882564 -0.41469419 -0.33076522] 1.220703125e-05 450.99299287332735
[-0.72956408  0.1463699  -0.40796775 -0.34585906] 1.220703125e-05 450.2132544185683
[-0.73403682  0.14551224 -0.4046298  -0.35349958] 1.220703125e-05 449.8241596211096
[-0.74142769  0.14441659 -0.3991371  -0.36639042] 2.44140625e-05 449.1857586596199
[-0.75536938  0.14125184 -0.38940957 -0.39165278] 1.220703125e-05 447.99865498499844
[-0.75817767  0.14126407 -0.38730084 -0.39689288] 2.44140625e-05 447.74014107114476
[-0.7621022   0.14067206 -0.38454577 -0.40430848] 1.220703125e-05 447.4107870692893
[-0.76566228  0.13942582 -0.38226325 -0.41113122] 1.220703125e-05 447.09191402611606
[-0.76586722  0.13933638 -0.38213728 -0.41152676] 1.220703125e-05 447.0797746548176
[-0.76642931  0.13971568 -0.38161273 -0.41261017] 2.44140625e-05 447.0171162164562
[-0.76917338  0.1392415  -0.37972231 -0.4179428 ] 1.220703125e-05 446.7775

I ran the algorithm four times. Here's the learned weights.

In [ ]:
# [-1.63963568  0.11244618 -0.25229998 -1.83450268]
# [-1.43047609  0.11237203 -0.25146534 -1.86348854]
# [-0.90529042  0.11226296 -0.24705779 -2.05801113]
# [-1.13011592  0.11231782 -0.24929065 -1.95950121]